In [1]:
import numpy as np

In [ ]:
M = 10 # num agents
T = 8 # timesteps
R = 0.5 # confidence bound
min_age = 16
max_age = 81
op_drift = 0.01

youth_min_op = 0
youth_max_op = 0.4

In [3]:
x = np.random.rand(M) # init all agents to [0,1]
age_of_x = np.random.randint(min_age,max_age, size=M) 

In [4]:
def get_A(x):
    X = np.array([x]*M)
    mask = abs(X - X.T) < R
    A = np.zeros(M)
    for i in range(M):
        A[i] = np.mean(x[mask[i]])
    return(A)

In [5]:
X = np.zeros((T,M))
X_age = np.zeros((T,M))
X[0,:] = x
X_age[0,:] = age_of_x
for t in range(T-1):
    deaths = X_age[t,:] > max_age
    num_deaths = sum(deaths)
    X_age[t, deaths] = min_age
    if(num_deaths > 0):
        print(num_deaths)
    X[t,:][deaths] = (youth_max_op - youth_min_op) * np.random.random(num_deaths) + youth_min_op  # resurrections
    
    X[t+1,:] = np.minimum(get_A(X[t,:]) + op_drift, 1.0)
    X_age[t+1,:] = X_age[t,:] + 1
X

array([[0.72672704, 0.71782838, 0.27403494, 0.16255486, 0.94045222,
        0.54445913, 0.92028334, 0.1688924 , 0.96653894, 0.88710948],
       [0.75717918, 0.75717918, 0.44241612, 0.29748533, 0.82477122,
        0.64088807, 0.82477122, 0.29748533, 0.82477122, 0.82477122],
       [0.65917181, 0.65917181, 0.65917181, 0.54210554, 0.74709343,
        0.65917181, 0.74709343, 0.54210554, 0.74709343, 0.74709343],
       [0.6809272 , 0.6809272 , 0.6809272 , 0.6809272 , 0.6809272 ,
        0.6809272 , 0.6809272 , 0.6809272 , 0.6809272 , 0.6809272 ],
       [0.6909272 , 0.6909272 , 0.6909272 , 0.6909272 , 0.6909272 ,
        0.6909272 , 0.6909272 , 0.6909272 , 0.6909272 , 0.6909272 ],
       [0.7009272 , 0.7009272 , 0.7009272 , 0.7009272 , 0.7009272 ,
        0.7009272 , 0.7009272 , 0.7009272 , 0.7009272 , 0.7009272 ],
       [0.7109272 , 0.7109272 , 0.7109272 , 0.7109272 , 0.7109272 ,
        0.7109272 , 0.7109272 , 0.7109272 , 0.7109272 , 0.7109272 ],
       [0.7209272 , 0.7209272 , 0.7209272

In [6]:
X_age

array([[72., 20., 40., 45., 52., 72., 27., 61., 22., 45.],
       [73., 21., 41., 46., 53., 73., 28., 62., 23., 46.],
       [74., 22., 42., 47., 54., 74., 29., 63., 24., 47.],
       [75., 23., 43., 48., 55., 75., 30., 64., 25., 48.],
       [76., 24., 44., 49., 56., 76., 31., 65., 26., 49.],
       [77., 25., 45., 50., 57., 77., 32., 66., 27., 50.],
       [78., 26., 46., 51., 58., 78., 33., 67., 28., 51.],
       [79., 27., 47., 52., 59., 79., 34., 68., 29., 52.]])